In [ ]:
import os
import h5py, tifffile
import numpy as np
import random, cv2
from tqdm import tqdm

os.chdir("../scripts/data_preparation")  
label_shape = (100, 3)

#### Setup
This cell should contain everything that you might need to change, with the one exception being adding datasets, in which case, in cell 3 you'd need to update the filepaths

In [ ]:
base_folder = "/home/brian/data4/brian/PBnJ/jelly_h5s/RFamide/mixed_datasets"

raw = True # If true, we will make the h5s from raw tif files provided
frames_per_dataset = 100 # How many frames from each dataset we should sample

add_extra = False # Should we add more frames from an nrrd to augment the labeled frames
num_frames_to_add = 500 # If we add extra frames, how many should we add
manually_selected_frames = [] # A list of the frames you want to make sure get included from this dataset
padding = "double" # "single" or "double" - Do you want to pad on both the top and bottom or just the bottom
side_len = 1024 # What the cropped image size should be (enforcing square) should prob be 1024, 1080, or 1200

augmentation = None # Options are "log" "sqrt" None

assert padding in ["single", "double"], f"Padding must be 'single' or 'double', not {padding}"

## Make the dataset from scratch if needed

Assumes you want all moving to one fixed frame

TODO: Reconsider this

In [ ]:
if raw:
    assert frames_per_dataset
    n_frames = 2000 # How many frames are in each tif file
    channel = 1 # Which channel to use (1 in RFa, 0 in normal - check)
    input_files = {
                    # "/storage/fs/store1/brian/swimming_videos_RFa/Folder_20250219110008_RFa_swim" : [
                    #                 "20250219_Experiment_01_0-1999.tif",
                    #                 "20250219_Experiment_01_2000-3999.tif",
                    #                 "20250219_Experiment_01_4000-5999.tif",
                    #                 "20250219_Experiment_01_8000-9999.tif",
                    #                 "20250219_Experiment_01_6000-7999.tif"],
                    "/storage/fs/store1/brian/swimming_videos_RFa/Folder_20250219120831_RFa": [
                                    "RIG_20250219_Experiment_01_0-1999.tif",
                                    "RIG_20250219_Experiment_01_2000-3999.tif",
                                    "RIG_20250219_Experiment_01_4000-5999.tif",
                                    "RIG_20250219_Experiment_01_6000-7999.tif",
                                    "RIG_20250219_Experiment_01_8000-9999.tif"],
                    # "/storage/fs/store1/brian/swimming_videos_RFa/Folder_20250214153740_RFa": [
                    #                 "20250214_Experiment_01_0-1999.tif",
                    #                 "20250214_Experiment_01_2000-3999.tif",
                    #                 "20250214_Experiment_01_4000-5999.tif"],
                    }

    unlabs = np.full((30, 2), -1)

    with h5py.File(os.path.join(base_folder, "moving_images.h5"), 'w-') as h5m, h5py.File(os.path.join(base_folder, "fixed_images.h5"), 'w-') as h5f, h5py.File(
            os.path.join(base_folder, "moving_labels.h5"), 'w-') as h5ml, h5py.File(os.path.join(base_folder, "fixed_labels.h5"), 'w-') as h5fl:
        dataset_ind = 0
        for dataset_name, file_list in tqdm(input_files.items()):
            file_list = [os.path.join(dataset_name, f) for f in file_list]

            dataset_name = os.path.basename(dataset_name)

            print(f"Processing {dataset_name}...")
            file_list = list(file_list)
            random.shuffle(file_list)

            total_frames = len(file_list) * n_frames
            if frames_per_dataset > total_frames:
                raise ValueError(f"Requested {frames_per_dataset} frames, but only {total_frames} available in {dataset_name}.")

            # Sample global frame indices
            selected_global_indices = sorted(random.sample(range(total_frames), frames_per_dataset))
            selected_global_indices += manually_selected_frames
            with open(os.path.join(base_folder, "frame_log.txt"), 'a') as f: # Save a log of the indices
                f.write(f"# {dataset_name}\n")
                for idx in selected_global_indices:
                    f.write(f"{idx}\n")

            # Iterate and extract frames
            current_global_index = 0
            saved_count = 0
            for tif_path in file_list:
                start, end = map(int, tif_path.split("_")[-1].replace(".tif", "").split("-"))

                with tifffile.TiffFile(tif_path) as tif:
                    arr = tif.asarray()
                    if arr.ndim != 4:
                        raise ValueError(f"Expected shape (T, C, H, W) but got {arr.shape} in {tif_path}")
                    
                    if start == 0:
                        # Save the fixed image
                        h5f.create_dataset(f"{dataset_ind}_0to{dataset_ind}_0", data=arr[0, channel])
                        h5fl.create_dataset(f"{dataset_ind}_0to{dataset_ind}_0", data=unlabs)

                    for idx in selected_global_indices:
                        if idx >= start and idx <= end:
                            local_idx = idx - start
                            frame = arr[local_idx, channel]  # shape (H, W)
                            ds_name = f"{dataset_ind}_{idx}to{dataset_ind}_0" # We're going to add the first number to diferentiate the datasets
                            h5m.create_dataset(ds_name, data=frame)
                            h5ml.create_dataset(ds_name, data=unlabs)

                            saved_count += 1

            dataset_ind += 1
            print(f"Saved {saved_count} frames for {dataset_name}")

            

  0%|          | 0/1 [00:00<?, ?it/s]

Processing Folder_20250219120831_RFa...


100%|██████████| 1/1 [09:33<00:00, 573.34s/it]

Saved 200 frames for Folder_20250219120831_RFa


# Clone fixed image to train
Start with images (formatted as just 1 fixed image from Weissbourd)

In [4]:
old_fixed_h5 = os.path.join(base_folder, "fixed_images.h5")
new_fixed_h5 = os.path.join(base_folder, "fixed_fixed_images.h5")
moving_h5 = os.path.join(base_folder, "moving_images.h5")
with h5py.File(old_fixed_h5, 'r') as f, h5py.File(moving_h5, 'r') as g, h5py.File(new_fixed_h5, 'w-') as fo:
    if len(f.keys()) == 1:
        img = f[list(f.keys())[0]][:]
        for prob in g.keys():
            fo.create_dataset(prob, data = img)
    else:
        assert len(set([f.split("_")[-1] for f in f.keys()])) == 1, "Expected a file with either one image or one image per dataset"
        base_imgs = {}
        for im in f.keys():
            base_imgs[im.split("_")[0]] = f[im][:]
        for prob in g.keys():
            fo.create_dataset(prob, data = base_imgs[prob.split("_")[0]])

Then do the same for labels

In [5]:
old_fixed_h5 = os.path.join(base_folder, "fixed_labels.h5")
new_fixed_h5 = os.path.join(base_folder, "fixed_fixed_labels.h5")
moving_h5 = os.path.join(base_folder, "moving_labels.h5")

with h5py.File(old_fixed_h5, 'r') as f, h5py.File(moving_h5, 'r') as g, h5py.File(new_fixed_h5, 'w-') as fo:
    if len(f.keys()) == 1:
        labs = f[list(f.keys())[0]][:]
        for prob in g.keys():
            fo.create_dataset(prob, data = labs)
    else:
        assert len(set([f.split("_")[-1] for f in f.keys()])) == 1, "Expected a file with either one image or one image per dataset"
        base_labs = {}
        for im in f.keys():
            base_labs[im.split("_")[0]] = f[im][:]
        for prob in g.keys():
            fo.create_dataset(prob, data = base_labs[prob.split("_")[0]])

Now add a third column so the labels have three dimensions

In [6]:
old_fixed_h5 = os.path.join(base_folder, "fixed_fixed_labels.h5")
new_fixed_h5 = os.path.join(base_folder, "fixed_fixed_fixed_labels.h5")
with h5py.File(old_fixed_h5, 'r') as f:
    assert f[list(f.keys())[0]][:].shape[1] == 2, "This is meant to expand the labels from 2D to 3D"
    with h5py.File(new_fixed_h5, 'w-') as g:
        for prob in f.keys():
            labs = f[prob][:]
            labs = np.concatenate((labs, np.zeros((labs.shape[0], 1))), 1)
            labs = np.pad(labs, [(0,label_shape[0] - labs.shape[0]), (0,0)], "constant", constant_values=-1)
            g.create_dataset(prob, data = labs)

In [7]:
old_moving_h5 = os.path.join(base_folder, "moving_labels.h5")
new_moving_h5 = os.path.join(base_folder, "fixed_moving_labels.h5")
with h5py.File(old_moving_h5, 'r') as f:
    assert f[list(f.keys())[0]][:].shape[1] == 2, "This is meant to expand the labels from 2D to 3D"
    with h5py.File(new_moving_h5, 'w-') as g:
        for prob in f.keys():
            labs = f[prob][:]
            labs = np.concatenate((labs, np.zeros((labs.shape[0], 1))), 1)
            labs = np.pad(labs, [(0,label_shape[0] - labs.shape[0]), (0,0)], "constant", constant_values=-1)
            g.create_dataset(prob, data = labs)

And add a third dim for the images

In [8]:
old_fixed_h5 = os.path.join(base_folder, "fixed_fixed_images.h5")
new_fixed_h5 = os.path.join(base_folder, "fixed_fixed_fixed_images.h5")
with h5py.File(old_fixed_h5, 'r') as f:
    assert len(f[list(f.keys())[0]][:].shape) == 2, "This is meant to expand the images from 2D to 3D"
    with h5py.File(new_fixed_h5, 'w-') as g:
        for prob in f.keys():
            img = f[prob][:]
            img = np.expand_dims(img, 2)
            g.create_dataset(prob, data = img)

In [9]:
old_moving_h5 = os.path.join(base_folder, "moving_images.h5")
new_moving_h5 = os.path.join(base_folder, "fixed_moving_images.h5")
with h5py.File(old_moving_h5, 'r') as f:
    assert len(f[list(f.keys())[0]][:].shape) == 2, "This is meant to expand the images from 2D to 3D"
    with h5py.File(new_moving_h5, 'w-') as g:
        for prob in f.keys():
            img = f[prob][:]
            img = np.expand_dims(img, 2)
            g.create_dataset(prob, data = img)

# Take unlabeled frames and add to a labeled h5

In [10]:
if add_extra:

    old_img_fixed_h5 = "/home/brian/data4/brian/PBnJ/jelly_h5s/full_lab_movds/fixed_images.h5"
    new_img_fixed_h5 = "/home/brian/data4/brian/PBnJ/jelly_processed_data/mixed_lab_h5/fixed_images.h5"

    old_label_fixed_h5 = "/home/brian/data4/brian/PBnJ/jelly_h5s/full_lab_movds/fixed_labels.h5"
    new_label_fixed_h5 = "/home/brian/data4/brian/PBnJ/jelly_processed_data/mixed_lab_h5/fixed_labels.h5"

    old_img_moving_h5 = "/home/brian/data4/brian/PBnJ/jelly_h5s/full_lab_movds/moving_images.h5"
    new_img_moving_h5 = "/home/brian/data4/brian/PBnJ/jelly_processed_data/mixed_lab_h5/moving_images.h5"

    old_label_moving_h5 = "/home/brian/data4/brian/PBnJ/jelly_h5s/full_lab_movds/moving_labels.h5"
    new_label_moving_h5 = "/home/brian/data4/brian/PBnJ/jelly_processed_data/mixed_lab_h5/moving_labels.h5"

    frame_folder = "/home/brian/data4/brian/PBnJ/jelly_centroid_prep/zoomed_in_vid"


    frame_names = [
        os.path.splitext(p)[0] for p in os.listdir(frame_folder)
        if os.path.splitext(p)[-1] in [".jpg"]
    ]

    probs = []

    for i in range(num_frames_to_add):
        probs.append(tuple(random.sample(frame_names, 2)))

    # old_mean = 0
    old_max = 0
    # num = 0

    with h5py.File(new_img_fixed_h5, 'w-') as nif:
        with h5py.File(new_label_fixed_h5, 'w-') as nlf:
            with h5py.File(old_img_fixed_h5, 'r') as oif:
                with h5py.File(old_label_fixed_h5, 'r') as olf:
                    for prob in oif.keys():
                        img = oif[prob][:]
                        labs = olf[prob][:]

                        # old_mean += np.mean(img)
                        old_max = max(old_max, np.max(img))
                        # num += 1

                        nif.create_dataset(prob, data = img, dtype=float)
                        nlf.create_dataset(prob, data = labs, dtype=np.float32)
            
            # old_mean = old_mean / num
            scale_factor = old_max / 255 # Photo max val
            
            unlabeled_labs = np.ones_like(labs, dtype=np.float32) * -1
            for fixed, moving in probs:
                prob = f"{moving}to{fixed}"
                img = cv2.imread(os.path.join(frame_folder, fixed + ".jpg"))[:,:,0:1]
                nif.create_dataset(prob, data = img * scale_factor, dtype=float)
                nlf.create_dataset(prob, data = unlabeled_labs, dtype=np.float32)





    with h5py.File(new_img_moving_h5, 'w-') as nim:
        with h5py.File(new_label_moving_h5, 'w-') as nlm:
            with h5py.File(old_img_moving_h5, 'r') as oim:
                with h5py.File(old_label_moving_h5, 'r') as olm:
                    for prob in oim.keys():
                        img = oim[prob][:]
                        labs = olm[prob][:]
                        nim.create_dataset(prob, data = img, dtype=float)
                        nlm.create_dataset(prob, data = labs, dtype=np.float32)

            for fixed, moving in probs:
                prob = f"{moving}to{fixed}"
                img = cv2.imread(os.path.join(frame_folder, moving + ".jpg"))[:,:,0:1]
                nim.create_dataset(prob, data = img * scale_factor, dtype=float)
                nlm.create_dataset(prob, data = unlabeled_labs, dtype=np.float32)

## Crop

In [11]:
in_dir = base_folder
out_dir = os.path.join(base_folder, "cropped")

crop_shape = np.array((side_len, side_len, 1))

os.makedirs(out_dir, exist_ok=True)

with h5py.File(os.path.join(in_dir, "fixed_moving_images.h5"), 'r') as imv,  h5py.File(os.path.join(in_dir, "fixed_moving_labels.h5"), 'r') as lmv, h5py.File(
    os.path.join(out_dir, "moving_images.h5"), 'w-') as oimv,  h5py.File(os.path.join(out_dir, "moving_labels.h5"), 'w-') as olmv:
    with h5py.File(os.path.join(in_dir, "fixed_fixed_fixed_images.h5"), 'r') as ifx,  h5py.File(os.path.join(in_dir, "fixed_fixed_fixed_labels.h5"), 'r') as lfx, h5py.File(
        os.path.join(out_dir, "fixed_images.h5"), 'w-') as oifx,  h5py.File(os.path.join(out_dir, "fixed_labels.h5"), 'w-') as olfx:
        for prob in imv.keys():
            img = imv[prob][:]
            imgF = ifx[prob][:]

            crop_offset = (img.shape - crop_shape) / 2
            assert np.all(crop_offset == crop_offset.astype(int))
            crop_offset = crop_offset.astype(int)

            ## Crop the images
            img = img[crop_offset[0]:crop_offset[0] + crop_shape[0], crop_offset[1]:crop_offset[1] + crop_shape[1], crop_offset[2]:crop_offset[2] + crop_shape[2]]
            assert np.all(img.shape == crop_shape)

            imgF = imgF[crop_offset[0]:crop_offset[0] + crop_shape[0], crop_offset[1]:crop_offset[1] + crop_shape[1], crop_offset[2]:crop_offset[2] + crop_shape[2]]
            assert np.all(imgF.shape == crop_shape), imgF.shape

            ## Adjust the labels
            labs = lmv[prob][:]
            neg_ones = labs < 0
            labs = labs - crop_offset 

            labsF = lfx[prob][:]
            neg_onesF = labsF < 0
            labsF = labsF - crop_offset
            assert np.all(neg_ones == neg_onesF), "Fixed and Moving labels have a different number of non negative 1 labels"
            
            # Remove centroids that are cropped out of the image
            crop = np.logical_or(labs < 0, labs >= crop_shape)
            crop = np.max(crop, axis=-1)
            cropF = np.logical_or(labsF < 0, labsF >= crop_shape)
            cropF = np.max(cropF, axis=-1)
            crop = np.logical_or(crop, cropF) # If either the fixed or moving are out of bounds then exclude both

            labs[crop] = -1
            labsF[crop] = -1

            labs[neg_ones] = -1 # Retain -1s
            labsF[neg_ones] = -1 # Retain -1s

            # Double check that all of the out of frame centroids are gone
            assert np.all(np.logical_or(labs >= 0, labs == -1)), "The crop results in moving centroids that are under bounds"
            assert np.all(labs < crop_shape[0]), "The crop results in moving centroids that are over bounds"
            assert np.all(np.logical_or(labsF >= 0, labsF == -1)), "The crop results in fixed centroids that are under bounds"
            assert np.all(labsF < crop_shape[0]), "The crop results in fixed centroids that are over bounds"

            assert np.all((labs == -1) == (labsF == -1)), "The labels that are excluded are not the same between moving and fixed"


            oimv.create_dataset(prob, data = img)
            olmv.create_dataset(prob, data = labs)
            oifx.create_dataset(prob, data = imgF)
            olfx.create_dataset(prob, data = labsF)


## Add Padding

In [12]:
in_dir = os.path.join(base_folder, "cropped")
out_dir = os.path.join(base_folder, "cropped", "padded")

os.makedirs(out_dir, exist_ok=True)

if padding == "single":
    padding = np.array([[0,0],[0,0],[1,0]])
else:
    padding = np.array([[0,0],[0,0],[1,1]])

with h5py.File(os.path.join(in_dir, "moving_images.h5"), 'r') as imv,  h5py.File(os.path.join(in_dir, "moving_labels.h5"), 'r') as lmv:
    with h5py.File(os.path.join(out_dir, "moving_images.h5"), 'w-') as oimv,  h5py.File(os.path.join(out_dir, "moving_labels.h5"), 'w-') as olmv:
        for prob in imv.keys():
            img = imv[prob][:]
            # img = np.pad(img, padding, "constant", constant_values=0)
            img = np.pad(img, padding, "constant", constant_values=np.min(img))

            labs = lmv[prob][:]
            neg_ones = labs < 0
            labs = labs + padding[:, 0] 
            labs[neg_ones] = -1 # Retain -1s

            oimv.create_dataset(prob, data = img)
            olmv.create_dataset(prob, data = labs)



with h5py.File(os.path.join(in_dir, "fixed_images.h5"), 'r') as ifx,  h5py.File(os.path.join(in_dir, "fixed_labels.h5"), 'r') as lfx:
    with h5py.File(os.path.join(out_dir, "fixed_images.h5"), 'w-') as oifx,  h5py.File(os.path.join(out_dir, "fixed_labels.h5"), 'w-') as olfx:
        for prob in ifx.keys():
            img = ifx[prob][:]
            # img = np.pad(img, padding, "constant", constant_values=0)
            img = np.pad(img, padding, "constant", constant_values=np.min(img))

            labs = lfx[prob][:]
            neg_ones = labs < 0
            labs = labs + padding[:, 0] 
            labs[neg_ones] = -1 # Retain -1s
            
            oifx.create_dataset(prob, data = img)
            olfx.create_dataset(prob, data = labs)


## Ceiling
#### log

In [3]:
if augmentation == "log":

    in_dir = os.path.join(base_folder, "cropped", "padded")
    out_dir = os.path.join(base_folder, "cropped", "padded", "log_scaled")

    os.makedirs(out_dir, exist_ok=True)

    with h5py.File(os.path.join(in_dir, "moving_images.h5"), 'r') as imv,  h5py.File(os.path.join(in_dir, "moving_labels.h5"), 'r') as lmv:
        with h5py.File(os.path.join(out_dir, "moving_images.h5"), 'w-') as oimv,  h5py.File(os.path.join(out_dir, "moving_labels.h5"), 'w-') as olmv:
            for prob in imv.keys():
                img = imv[prob][:]
                labs = lmv[prob][:]
                
                img = np.log2(img - (np.min(img) - 1), dtype=np.float32)
                # img = np.log(img + 1, dtype=np.float32)

                oimv.create_dataset(prob, data = img)
                olmv.create_dataset(prob, data = labs)



    with h5py.File(os.path.join(in_dir, "fixed_images.h5"), 'r') as ifx,  h5py.File(os.path.join(in_dir, "fixed_labels.h5"), 'r') as lfx:
        with h5py.File(os.path.join(out_dir, "fixed_images.h5"), 'w-') as oifx,  h5py.File(os.path.join(out_dir, "fixed_labels.h5"), 'w-') as olfx:
            for prob in ifx.keys():
                img = ifx[prob][:]
                labs = lfx[prob][:]
                
                img = np.log2(img - (np.min(img) - 1), dtype=np.float32)
                # img = np.log(img + 1, dtype=np.float32)
                
                oifx.create_dataset(prob, data = img)
                olfx.create_dataset(prob, data = labs)


#### sqrt

In [ ]:
if augmentation == "sqrt":

    in_dir = os.path.join(base_folder, "cropped", "padded")
    out_dir = os.path.join(base_folder, "cropped", "padded", "sqrt_scaled")



    with h5py.File(os.path.join(in_dir, "moving_images.h5"), 'r') as imv,  h5py.File(os.path.join(in_dir, "moving_labels.h5"), 'r') as lmv:
        with h5py.File(os.path.join(out_dir, "moving_images.h5"), 'w-') as oimv,  h5py.File(os.path.join(out_dir, "moving_labels.h5"), 'w-') as olmv:
            for prob in imv.keys():
                img = imv[prob][:]
                labs = lmv[prob][:]
                
                img = np.sqrt(img - np.min(img), dtype=np.float32)

                oimv.create_dataset(prob, data = img)
                olmv.create_dataset(prob, data = labs)



    with h5py.File(os.path.join(in_dir, "fixed_images.h5"), 'r') as ifx,  h5py.File(os.path.join(in_dir, "fixed_labels.h5"), 'r') as lfx:
        with h5py.File(os.path.join(out_dir, "fixed_images.h5"), 'w-') as oifx,  h5py.File(os.path.join(out_dir, "fixed_labels.h5"), 'w-') as olfx:
            for prob in ifx.keys():
                img = ifx[prob][:]
                labs = lfx[prob][:]
                
                img = np.sqrt(img - np.min(img), dtype=np.float32)
                
                oifx.create_dataset(prob, data = img)
                olfx.create_dataset(prob, data = labs)


In [15]:
with h5py.File(os.path.join(in_dir, "moving_images.h5"), 'r') as f:
    probs = list(f.keys())
    print(probs)
    print(len(probs))

['0_1008to0_0', '0_102to0_0', '0_1098to0_0', '0_1101to0_0', '0_1110to0_0', '0_1122to0_0', '0_1204to0_0', '0_1225to0_0', '0_1234to0_0', '0_1279to0_0', '0_1394to0_0', '0_1424to0_0', '0_1438to0_0', '0_1595to0_0', '0_1600to0_0', '0_163to0_0', '0_1640to0_0', '0_1658to0_0', '0_1845to0_0', '0_1930to0_0', '0_1988to0_0', '0_198to0_0', '0_1991to0_0', '0_1to0_0', '0_2007to0_0', '0_2009to0_0', '0_2115to0_0', '0_2119to0_0', '0_2147to0_0', '0_2149to0_0', '0_2152to0_0', '0_2179to0_0', '0_2323to0_0', '0_2381to0_0', '0_2486to0_0', '0_2491to0_0', '0_2503to0_0', '0_2607to0_0', '0_2634to0_0', '0_2638to0_0', '0_2646to0_0', '0_2722to0_0', '0_2762to0_0', '0_2815to0_0', '0_281to0_0', '0_2916to0_0', '0_3061to0_0', '0_3072to0_0', '0_3106to0_0', '0_3113to0_0', '0_3144to0_0', '0_3283to0_0', '0_3297to0_0', '0_3299to0_0', '0_3334to0_0', '0_334to0_0', '0_3361to0_0', '0_3365to0_0', '0_336to0_0', '0_3387to0_0', '0_3429to0_0', '0_3462to0_0', '0_3504to0_0', '0_3623to0_0', '0_3625to0_0', '0_3665to0_0', '0_3671to0_0', '0_

# Split into training and val

In [16]:
val_probs = random.sample(probs, 30)
print(val_probs)

['0_647to0_0', '0_1640to0_0', '0_7681to0_0', '0_3959to0_0', '0_3954to0_0', '0_9233to0_0', '0_6258to0_0', '0_8105to0_0', '0_6647to0_0', '0_543to0_0', '0_575to0_0', '0_8926to0_0', '0_2916to0_0', '0_5863to0_0', '0_902to0_0', '0_3061to0_0', '0_1234to0_0', '0_9494to0_0', '0_6558to0_0', '0_710to0_0', '0_6467to0_0', '0_4684to0_0', '0_739to0_0', '0_3671to0_0', '0_2634to0_0', '0_4949to0_0', '0_1600to0_0', '0_5431to0_0', '0_3830to0_0', '0_3689to0_0']


In [ ]:
val_probs = ['0_647to0_0', '0_1640to0_0', '0_7681to0_0', '0_3959to0_0', '0_3954to0_0', '0_9233to0_0', '0_6258to0_0', '0_8105to0_0', '0_6647to0_0', '0_543to0_0', '0_575to0_0', '0_8926to0_0', '0_2916to0_0', '0_5863to0_0', '0_902to0_0', '0_3061to0_0', '0_1234to0_0', '0_9494to0_0', '0_6558to0_0', '0_710to0_0', '0_6467to0_0', '0_4684to0_0', '0_739to0_0', '0_3671to0_0', '0_2634to0_0', '0_4949to0_0', '0_1600to0_0', '0_5431to0_0', '0_3830to0_0', '0_3689to0_0']

if augmentation == None:
    base_dir = os.path.join(base_folder, "cropped", "padded")
elif augmentation == "sqrt":
    base_dir = os.path.join(base_folder, "cropped", "padded", "sqrt_scaled")    
elif augmentation == "log":
    base_dir = os.path.join(base_folder, "cropped", "padded", "log_scaled") 
else:
    raise ValueError(f"Unknown augmentation {augmentation}")

os.mkdir(os.path.join(base_dir, "train"))
os.mkdir(os.path.join(base_dir, "val"))

old_img_fixed_h5 = f"{base_dir}/fixed_images.h5"
train_img_fixed_h5 = f"{base_dir}/train/fixed_images.h5"
val_img_fixed_h5 = f"{base_dir}/val/fixed_images.h5"

old_label_fixed_h5 = f"{base_dir}/fixed_labels.h5"
train_label_fixed_h5 = f"{base_dir}/train/fixed_labels.h5"
val_label_fixed_h5 = f"{base_dir}/val/fixed_labels.h5"

old_img_moving_h5 = f"{base_dir}/moving_images.h5"
train_img_moving_h5 = f"{base_dir}/train/moving_images.h5"
val_img_moving_h5 = f"{base_dir}/val/moving_images.h5"

old_label_moving_h5 = f"{base_dir}/moving_labels.h5"
train_label_moving_h5 = f"{base_dir}/train/moving_labels.h5"
val_label_moving_h5 = f"{base_dir}/val/moving_labels.h5"


with h5py.File(train_img_fixed_h5, 'w-') as tif,  h5py.File(val_img_fixed_h5, 'w-') as vif:
    with h5py.File(train_label_fixed_h5, 'w-') as tlf,  h5py.File(val_label_fixed_h5, 'w-') as vlf:
        with h5py.File(old_img_fixed_h5, 'r') as oif:
            with h5py.File(old_label_fixed_h5, 'r') as olf:
                for prob in oif.keys():
                    img = oif[prob][:]
                    labs = olf[prob][:]
                    if prob in val_probs:
                        vif.create_dataset(prob, data = img)
                        vlf.create_dataset(prob, data = labs)
                    else:
                        tif.create_dataset(prob, data = img)
                        tlf.create_dataset(prob, data = labs)
        

with h5py.File(train_img_moving_h5, 'w-') as tim,  h5py.File(val_img_moving_h5, 'w-') as vim:
    with h5py.File(train_label_moving_h5, 'w-') as tlm,  h5py.File(val_label_moving_h5, 'w-') as vlm:
        with h5py.File(old_img_moving_h5, 'r') as oim:
            with h5py.File(old_label_moving_h5, 'r') as olm:
                for prob in oim.keys():
                    img = oim[prob][:]
                    labs = olm[prob][:]
                    if prob in val_probs:
                        vim.create_dataset(prob, data = img)
                        vlm.create_dataset(prob, data = labs)
                    else:
                        tim.create_dataset(prob, data = img)
                        tlm.create_dataset(prob, data = labs)

### Create empty ROIs

In [18]:
old_img_moving_h5 = f"{base_dir}/moving_images.h5"

train_roi_moving_h5 = f"{base_dir}/train/moving_rois.h5"
val_roi_moving_h5 = f"{base_dir}/val/moving_rois.h5"
train_roi_fixed_h5 = f"{base_dir}/train/fixed_rois.h5"
val_roi_fixed_h5 = f"{base_dir}/val/fixed_rois.h5"


with h5py.File(val_roi_fixed_h5, 'w-') as vrf,  h5py.File(val_roi_moving_h5, 'w-') as vrm:
# with h5py.File(train_roi_fixed_h5, 'w-') as trf,  h5py.File(train_roi_moving_h5, 'w-') as trm:
    with h5py.File(old_img_moving_h5, 'r') as oim:
        for prob in oim.keys():
            blank = np.zeros_like(oim[prob][:])
            vrf.create_dataset(prob, data = blank)
            vrm.create_dataset(prob, data = blank)
            # trf.create_dataset(prob, data = blank)
            # trm.create_dataset(prob, data = blank)